<h1> Notebook: [ Week #06: Part 2 - Ah Beng on a Job Hunt ]

The overall objective of this notebook is to create a multi-agent system using the CrewAI framework to assist job applicants in various stages of their job search and application process. 

- This system leverages specialized agents to analyze job postings, compile detailed personal and professional profiles, and tailor resumes to better match job requirements. 

- The agents should utilize any relevant tools to achieve their goals, providing a comprehensive solution to enhance the job application experience for candidates.

    - You are encouraged to visit the official documentation page site for CrewAI https://docs.crewai.com/ to discover and incorporate variety of techniques, including those that are not covered in the previous notebook.

> 💡 While you can prepare the code in Notebook, the end output is a Python script (or scripts) can run with a single click or via command line tool.

In [4]:
# Common imports
import os
from dotenv import load_dotenv
import json
import lolviz
import requests
# Import the key CrewAI classes
from crewai import Agent, Task, Crew

In [5]:
# Load the environment variables
# If the .env file is not found, the function will return `False
load_dotenv('.env')

True

In [6]:
# download the CV to local current directory
url_resume = "https://abc-notes.data.tech.gov.sg/resources/data/fake-cv.md"
response = requests.get(url_resume)

with open("fake-cv.md", 'wb') as f:
    f.write(response.content)
    f.close()
    print("Downloaded fake-cv.md")

Downloaded fake-cv.md


In [7]:
# import other necessary libraries you need

from crewai_tools import FileReadTool
from crewai_tools import WebsiteSearchTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

# Create the Agents

### Instruction

You will create new agents with specific roles and goals. 

`Minimally`, create three new agents: a Tech Job Researcher, a Personal Profiler, and a Resume Strategist for the job applicant.

1. **Tech Job Researcher**
    - **Role**: Tech Job Researcher
    - **Goal**: Perform thorough analysis on job postings to assist job applicants.
    - **Backstory**: As a Job Researcher, your expertise in navigating and extracting critical information from job postings is unparalleled. Your skills help identify the necessary qualifications and skills sought by employers, forming the foundation for effective application tailoring.


2. **Personal Profiler**
    - **Role**: Personal Profiler
    - **Goal**: Conduct comprehensive research on job applicants to help them stand out in the job market.
    - **Backstory**: Equipped with analytical prowess, you dissect and synthesize information from diverse sources to craft comprehensive personal and professional profiles, laying the groundwork for personalized resume enhancements.


3. **Resume Strategist**
    - **Role**: Resume Strategist
    - **Goal**: Discover the best ways to make a resume stand out in the job market.
    - **Backstory**: With a strategic mind and an eye for detail, you excel at refining resumes to highlight the most relevant skills and experiences, ensuring they resonate perfectly with the job's requirements.




In [8]:
# You can use this cell to create tools or functions that you will use in the main code
# Hint: You may want to use `FileReadTool` tool which can be used by agent/task to read the resume file

file_read_tool = FileReadTool(file_path='./fake-cv.md')
tool_webscape = ScrapeWebsiteTool()


In [9]:
# Create the agents here
researcher = Agent(
    role="Tech Job Researcher",
    goal="Perform thorough analysis on job postings to assist job applicants.",

    backstory="""\
    As a Job Researcher, your expertise in navigating and extracting critical information from job postings is unparalleled. 
    Your skills help identify the necessary qualifications and skills sought by employers, forming the foundation for effective application tailoring.""",
    tools= [tool_webscape],
    allow_delegation=False, 
	verbose=True, 
)

profiler = Agent(
    role="Personal Profiler",
    goal="Conduct comprehensive research on job applicants to help them stand out in the job market",

    backstory="""\
    Equipped with analytical prowess, you dissect and synthesize information from diverse sources to craft comprehensive personal and professional profiles, laying the groundwork for personalized resume enhancements.""",
    tools = [file_read_tool],
    allow_delegation=False, 
	verbose=True, 
)

resume_strategist = Agent(
    role="Resume Strategist",
    goal="Discover the best ways to make a resume stand out in the job market.",

    backstory="""\
    With a strategic mind and an eye for detail, you excel at refining resumes to highlight the most relevant skills and experiences, ensuring they resonate perfectly with the job's requirements.
""",

    allow_delegation=False, 
	verbose=True, 
)

# Create the Tasks

Here, you will create tasks for the agents defined earlier. 
Follow the steps below to create tasks for the Tech Job Researcher, Personal Profiler for Engineers, and Resume Strategist for Engineers.

1. **Define the task for the Tech Job Researcher Agent: Extract Job Requirements**
    - **Description**: Analyze the job posting URL provided (`{job_posting_url}`) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
    - **Expected Output**: A structured list of job requirements, including necessary skills, qualifications, and experiences.
    - **Agent**: [`researcher`]


2. **Define the task for the Personal Profiler for Engineers Agent: Compile Comprehensive Profile**
    - **Description**: Compile a detailed personal and professional profile based on the current CV.
    - **Expected Output**: A comprehensive profile document that includes skills, project experiences, contributions, interests, and communication style.
    - **Agent**: [`profiler`]

3. **Define the task for the Resume Strategist for Engineers Agent: Align Resume with Job Requirements**
    - **Description**: Using the profile and job requirements obtained from previous tasks, tailor the resume to highlight the most relevant areas. Employ tools to adjust and enhance the resume content. Make sure this is the best resume ever but don't make up any information. Update every section, including the initial summary, work experience, skills, and education. All to better reflect the candidate's abilities and how it matches the job posting.
    - **Expected Output**: An updated resume that effectively highlights the candidate's qualifications and experiences relevant to the job.
    - **Agent**: [`resume_strategist`]




In [10]:
task_extract = Task(
    description="""\
    Analyze the job posting URL provided (`{job_posting_url}`) to extract key skills, experiences, and qualifications required. 
    Use the tools to gather content and identify and categorize the requirements.""",

    expected_output="""\
    A structured list of job requirements, including necessary skills, qualifications, and experiences.""",
    agent=researcher,
    async_execution=True,
)

task_compile = Task(
    description="""\
    Compile a detailed personal and professional profile based on the current CV.""",

    expected_output="""\
    A comprehensive profile document that includes skills, project experiences, contributions, interests, and communication style.""",
    agent=profiler,
    async_execution=True,
)

task_align = Task(
    description="""\
    Using the profile and job requirements obtained from previous tasks, tailor the resume to highlight the most relevant areas. 
    Employ tools to adjust and enhance the resume content. Make sure this is the best resume ever but don't make up any information. 
    Update every section, including the initial summary, work experience, skills, and education. 
    All to better reflect the candidate's abilities and how it matches the job posting.""",

    expected_output="""\
    An updated resume that effectively highlights the candidate's qualifications and experiences relevant to the job.""",
    agent=resume_strategist,
    context=[task_extract, task_compile],
)

# Create the Crew

In [11]:
crew = Crew(
    agents=[researcher, profiler, resume_strategist],
    tasks=[task_extract, task_compile, task_align],
    verbose=True
)

# Running 

In [12]:
job_application_inputs = {
    'job_posting_url': 'https://www.mycareersfuture.gov.sg/job/information-technology/full-stack-developer-ntt-data-singapore-12059bf21549d1794e3535de365d0a77',
}


### this execution will take a few minutes to run
result = crew.kickoff(inputs=job_application_inputs)

# Agent: Tech Job Researcher
## Task:     Analyze the job posting URL provided (`https://www.mycareersfuture.gov.sg/job/information-technology/full-stack-developer-ntt-data-singapore-12059bf21549d1794e3535de365d0a77`) to extract key skills, experiences, and qualifications required. 
    Use the tools to gather content and identify and categorize the requirements.
# Agent: Personal Profiler
## Task:     Compile a detailed personal and professional profile based on the current CV.


# Agent: Tech Job Researcher
## Thought: I need to analyze the content of the job posting available at the provided URL to extract key skills, experiences, and qualifications required.
## Using tool: Read website content
## Tool Input: 
{
  "website_url": "https://www.mycareersfuture.gov.sg/job/information-technology/full-stack-developer-ntt-data-singapore-12059bf21549d1794e3535de365d0a77"
}
## Tool Output: 
MyCareersFuture Singapore


# Agent: Personal Profiler
## Thought: Thought: I need to start by obtaini